# Initial Setup

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

from utils.matchers import FeatureMatcher, MultipleInstanceMatcher
from utils.visualization import display_frames
from utils.utils import frame_generator

video_filename = './Data/Multiple View.avi'

reference_frame = cv2.cvtColor(cv2.imread('Data/ReferenceFrame.png', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
reference_mask = cv2.imread('Data/ObjectMask.PNG', cv2.IMREAD_GRAYSCALE)

ar_layer = cv2.cvtColor(cv2.imread('Data/AugmentedLayer.PNG', cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)[:,:640]
ar_mask = cv2.imread('Data/AugmentedLayerMask.PNG', cv2.IMREAD_GRAYSCALE)[:,:640]


#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

# Overview of the data

## Reference frame

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(reference_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## AR layer

In [ ]:
w, h, dpi = 1000, 500, 100
fig, ax = plt.subplots(ncols=2, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(ar_layer)
ax[0].set_title('AR layer')

ax[1].imshow(ar_mask, cmap='gray')
ax[1].set_title('Mask')

fig.tight_layout(pad=0.5)
plt.show()

## Video sequence

In [ ]:
display_frames(video_filename, starting_frame=0)

# Stitch AR layer onto reference frame

In [ ]:
ar_frame = reference_frame.copy()
ar_frame[ar_mask==255] = ar_layer[ar_mask==255]

w, h, dpi = 1200, 400, 100
fig, ax = plt.subplots(ncols=3, figsize=(w/dpi, h/dpi), dpi=dpi)

ax[0].imshow(reference_frame)
ax[0].set_title('Reference frame')

ax[1].imshow(ar_layer)
ax[1].set_title('AR layer')

ax[2].imshow(ar_frame)
ax[2].set_title('Combination')

fig.tight_layout(pad=0.5)
plt.show()

# Do this for all the video frames
This is done manually. The transformation should be the identity.

In [ ]:
#TODO: improve this and move it in utils

from IPython.display import clear_output

starting_frame = 100

for i, frame in enumerate(frame_generator(video_filename)):
    if i < starting_frame:
        continue
    matcher = FeatureMatcher(reference_frame, frame)
    matcher.find_matches()
    H, _ = matcher.get_homography()

    h, w = ar_layer.shape[0], ar_layer.shape[1]
    ar_layer_warped = cv2.warpPerspective(ar_layer, H, dsize=(w, h))
    ar_mask_warped = cv2.warpPerspective(ar_mask, H, dsize=(w, h))

    ar_frame = frame.copy()
    ar_frame[ar_mask_warped==255] = ar_layer_warped[ar_mask_warped==255]

    clear_output(wait=True)

    w, h, dpi = 1200, 400, 100
    fig, ax = plt.subplots(ncols=3, figsize=(w/dpi, h/dpi), dpi=dpi)

    ax[0].imshow(frame)
    ax[0].set_title('Original frame')

    ax[1].imshow(ar_layer_warped)
    ax[1].set_title('AR layer')

    ax[2].imshow(ar_frame)
    ax[2].set_title('Combination')

    fig.tight_layout(pad=0.5)
    plt.show()